# An example of using BRepNet embeddings for face similarity matching
This example notebook shows how the embeddings generated by the BRepNet model can be used for other tasks like selecitng similar faces.

In [1]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import os
if os.path.isfile('../models/brepnet.py'):
    # We are in the notebooks directory.  Change to the root
    os.chdir('../')

In [2]:
from pathlib import Path

# This code allows you to evaluate a pre-trained model for all step files in a folder
from eval.evaluate_folder import evaluate_folder

# This viewer allows you to visualize the results
from visualization.jupyter_segmentation_viewer import JupyterSegmentationViewer

from visualization.save_images_of_similar_solids import SimilarSolidImageSaver
from occwl.solid import Solid

Now we would like to evaluate the model for some example files.  We can do this using `eval/evaluate_folder.py`.  We need to supply the script with the path to the step files to evaluate, the feature standadization and the pretrained model to use.


In [3]:
# Here is the path to some example step files for us to convert
step_folder = Path("./example_files/step_examples")

# You may want to run this on the entire extended STEP dataset which you can doanload from 
# https://fusion-360-gallery-dataset.s3.us-west-2.amazonaws.com/segmentation/s2.0.0/s2.0.0_extended_step.zip
# step_folder = Path("/path/to/s2.0.0_extended_step/breps/step/")

# We will also need to know the feature standardization for the dataset used to train the model
# This is found in the dataset file created by pipeline/build_dataset_file.py or pipeline/quickstart.py
feature_standardization = Path("./example_files/feature_standardization/s2.0.0_step_all_features.json")

# Here is the path to a pretrained model
pretrained_model = Path("./example_files/pretrained_models/pretrained_s2.0.0_extended_step_uv_net_features_0816_183419.ckpt")

# Now we can evaluate the model on these step files.
# Depending on your system you may see pytorch lightning warning you  
# GPUs are unused and more worker threads could be used in the dataloader.
# The default options here are intended to work on a minimal system.
evaluate_folder(step_folder, feature_standardization, model=pretrained_model)

0it [00:00, ?it/s]
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Completed pipeline/extract_feature_data_from_step.py
Using labels from example_files/step_examples/temp_working


/home/lambouj/anaconda3/envs/brepnet/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
/home/lambouj/anaconda3/envs/brepnet/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/Chamfer_iou': 0.8418079018592834,
 'test/CutEnd_iou': 0.7290322780609131,
 'test/CutSide_iou': 0.8102856874465942,
 'test/ExtrudeEnd_iou': 0.7076271176338196,
 'test/ExtrudeSide_iou': 0.8099502325057983,
 'test/Fillet_iou': 0.9281437397003174,
 'test/RevolveEnd_iou': 0.8999999761581421,
 'test/RevolveSide_iou': 0.7386363744735718,
 'test/accuracy': 0.8996027708053589,
 'test/mean_iou': 0.8081854581832886}
--------------------------------------------------------------------------------


In [4]:
step_file_stems = [ f.stem for f in step_folder.glob("*.stp")]
print(f"We found {len(step_file_stems)} example files")

We found 25 example files


Choose the model you would like to use as the query

In [5]:
query_index = 17

Now you will need to select some faces on the solid to continue.  
Double click on each face to select it

In [13]:
file_stem = step_file_stems[query_index]
print(f"Viewing example {file_stem}")
viewer = JupyterSegmentationViewer(file_stem, step_folder, seg_folder=step_folder)
viewer.view_solid()

Viewing example 21242_6c2af7c2_7


In [14]:
assert len(viewer.selection_list) > 0, "Please select some faces on the solid"

In [15]:
# Load the embeddings for the query solid
embeddings_folder = step_folder / "temp_working/embeddings"
embeddings_pathname = embeddings_folder / (file_stem + ".embeddings")
embeddings = np.loadtxt(embeddings_pathname)
assert embeddings.shape[0] == len(viewer.entity_mapper.face_map), "Embedding size doesn't match solid"
selected_face_embeddings = embeddings[viewer.selection_list]

In [16]:
# Loop over all solids and find the distance from each face of each solid
# to each query face
all_embeddings = list(embeddings_folder.glob("*.embeddings"))
min_dists_for_each_face = []
sum_min_dists_for_each_solid = []
for embedding_file in all_embeddings:
    target_file_stem = embedding_file.stem
    
    # Load the embeddings for the target file
    embeddings = np.loadtxt(embedding_file)
    if len(embeddings.shape) == 1:
        embeddings = np.expand_dims(embeddings, axis=0)
        
    min_dists = []
    min_dists_for_each_face_in_this_solid = []
    for face_embedding in selected_face_embeddings:
        face_to_embeddings = embeddings-face_embedding
        dist = np.linalg.norm(face_to_embeddings, axis=1)
        
        # Here we compute the distance from a given query face
        # to each face in the solid
        min_dists_for_each_face_in_this_solid.append(dist)
        
        # Then is metric is the distance from that given query face to
        # the most similar face in the solid
        min_dists.append(np.min(dist))
    
    # Now we need some way to say which solids is the best match over all
    # We do this by summing the minimum distances to all the faces 
    sum_dists = np.sum(np.stack(min_dists), axis=0)
    sum_min_dists_for_each_solid.append(sum_dists)
    
    # For display we also keep track of the min distance to each face
    min_dists_for_each_face_in_this_solid = np.min(np.stack(min_dists_for_each_face_in_this_solid), axis=0)
    min_dists_for_each_face.append(min_dists_for_each_face_in_this_solid)
    
sum_min_dists_for_each_solid = np.array(sum_min_dists_for_each_solid)      

In [17]:
# Now we will search for the top k best matches
k = 5
indices_of_smallest = np.argpartition(sum_min_dists_for_each_solid, kth=range(k))[:k]

# We want to show how close each face in each solid is
# To do this we want some kind of range which is computed here
all_dists_top_k = []
for index in indices_of_smallest:
    all_dists_top_k.append(min_dists_for_each_face[index])
all_dists_top_k = np.concatenate(all_dists_top_k)
interval = [all_dists_top_k.min(), all_dists_top_k.max()]
print(f"Interval {interval}")

Interval [0.0, 18.090275079197]


In [18]:
# Now as a sanity check we always expect to find the query as the best 
# match.  The other matching faces (red) should have a similar shape
for i, index in enumerate(indices_of_smallest):
    print(index)
    close_file_stem = all_embeddings[index].stem
    print(f"Close file {close_file_stem}")
    close_viewer = JupyterSegmentationViewer(close_file_stem, step_folder)
    dists_to_view = min_dists_for_each_face[index]
    close_viewer.display_faces_with_heatmap(dists_to_view, interval)

7
Close file 21242_6c2af7c2_7


22
Close file 56436_2a8fc254_3


3
Close file 24051_4852a192_5


5
Close file 44647_d83249a9_0


15
Close file 21492_8bd34fc1_15
